# Imports

In [1]:
import findspark

findspark.init('C:/spark')

In [74]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# Create a session

In [3]:
spark = SparkSession.builder.appName('tree').getOrCreate()

# Read data

In [35]:
data = spark.read.csv('../../data/College.csv',
                      inferSchema=True,
                      header=True)

In [36]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [37]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

# Data preprocessing with a Pipeline

In [38]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [39]:
# Selecting columns for assembler
inputCols = data.columns
inputCols.remove('School')
inputCols.remove('Private')

In [40]:
assembler = VectorAssembler(inputCols=inputCols, outputCol='features')
indexer = StringIndexer(inputCol='Private', outputCol='PrivateIndex')

In [41]:
pipeline = Pipeline(stages=[
    assembler,
    indexer
])

In [42]:
preprocessing_model = pipeline.fit(data)

In [43]:
preprocessed_data = preprocessing_model.transform(data)

In [44]:
preprocessed_data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- PrivateIndex: double (nullable = false)



In [45]:
final_data = preprocessed_data.select('features', 'PrivateIndex')

In [46]:
final_data.show()

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
|[587.0,479.0,158....|         0.0|
|[353.0,340.0,103....|         0.0|
|[1899.0,1720.0,48...|         0.0|
|[1038.0,839.0,227...|         0.0|
|[582.0,498.0,172....|         0.0|
|[1732.0,1425.0,47...|         0.0|
|[2652.0,1900.0,48...|         0.0|
|[1179.0,780.0,290...|         0.0|
|[1267.0,1080.0,38...|         0.0|
|[494.0,313.0,157....|         0.0|
|[1420.0,1093.0,22...|         0.0|
|[4302.0,992.0,418...|         0.0|
|[1216.0,908.0,423...|         0.0|
|[1130.0,704.0,322...|         0.0|
|[3540.0,2001.0,10...|         1.0|
+--------------------+------------+
only showing top 20 rows



In [47]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

# Modeling

In [69]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex', featuresCol='features', numTrees=150)
gbt = GBTClassifier(labelCol='PrivateIndex', featuresCol='features')

In [70]:
binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex', metricName='areaUnderROC')

In [76]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex', metricName='accuracy')

In [77]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [78]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [79]:
dtc_roc_score = binary_eval.evaluate(dtc_preds)
print(f"DTC ROC Score: {dtc_roc_score}")

rfc_roc_score = binary_eval.evaluate(rfc_preds)
print(f"RFC ROC Score: {rfc_roc_score}")

gbt_roc_score = binary_eval.evaluate(gbt_preds)
print(f"GBT ROC Score: {gbt_roc_score}")

DTC ROC Score: 0.7753629932985853
RFC ROC Score: 0.9762658227848111
GBT ROC Score: 0.9555100521221146


In [81]:
dtc_acc_score = acc_eval.evaluate(dtc_preds)
print(f"DTC Accuracy Score: {dtc_acc_score}")

rfc_acc_score = acc_eval.evaluate(rfc_preds)
print(f"RFC Accuracy Score: {rfc_acc_score}")

gbt_acc_score = acc_eval.evaluate(gbt_preds)
print(f"GBT Accuracy Score: {gbt_acc_score}")

DTC Accuracy Score: 0.8938053097345132
RFC Accuracy Score: 0.915929203539823
GBT Accuracy Score: 0.9026548672566371
